# Weighted Cost Function

Shows how to use the cost function requested in [issue #84](https://github.com/EthanJamesLew/AutoKoopman/issues/84).

In [ ]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("..")
# this is the convenience function
from autokoopman import auto_koopman

In [ ]:
# for a complete example, let's create an example dataset using an included benchmark system
import autokoopman.benchmark.fhn as fhn
fhn = fhn.FitzHughNagumo()
training_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.0, high=2.0, size=(10, 2)),
    tspan=[0.0, 10.0],
    sampling_period=0.1
)

In [ ]:
# create trajectories as numpy array and create a weights array
trajectories = []
weights = []

# create weights for every time point
for idx, traj in enumerate(training_data):
    trajectories.append(traj.states)
    
    # uniform weights
    #weights.append(np.ones(len(traj.states)) / len(traj.states) )

    # distance weights
    #weights.append(traj.norm().states / (np.max(traj.norm().states) * len(traj.states)) )
    
    # individual state weights
    #w = np.abs(traj.states) / 10.0
    #w[:, 0] = 0.0
    #w = np.ones(traj.states.shape) / 10.0
    w = np.sum(traj.abs().states, axis=1)
    weights.append(w)

# you can also use a dict to name the trajectories if using TrajectoriesData (numpy arrays are named by their index number)
#weights = {idx: w for idx, w in enumerate(weights)}

In [ ]:
# learn model from data
experiment_results = auto_koopman(
    trajectories,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type="rff",         # use Random Fourier Features Observables
    cost_func="weighted",   # use "weighted" cost function
    learning_weights=weights, # weight the eDMD algorithm objectives
    scoring_weights=weights, # pass weights as required for cost_func="weighted"
    opt="grid",             # grid search to find best hyperparameters
    n_obs=200,              # maximum number of observables to try
    max_opt_iter=200,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# learn model from data
experiment_results = auto_koopman(
    trajectories,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type="rff",         # use Random Fourier Features Observables
    cost_func="weighted",   # use "weighted" cost function
    scoring_weights=weights, # pass weights as required for cost_func="weighted"
    opt="grid",             # grid search to find best hyperparameters
    n_obs=200,              # maximum number of observables to try
    max_opt_iter=200,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# view our custom weighted cost
experiment_results

In [ ]:
# get the model from the experiment results
model = experiment_results['tuned_model']

# simulate using the learned model
iv = [0.5, 0.1]
trajectory = model.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)

In [ ]:
# simulate the ground truth for comparison
true_trajectory = fhn.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)

plt.figure(figsize=(10, 6))

# plot the results
plt.plot(*trajectory.states.T, label='Trajectory Prediction')
plt.plot(*true_trajectory.states.T, label='Ground Truth')

plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.grid()
plt.legend()
plt.title("FHN Test Trajectory Plot")
plt.show()